In [81]:
from share import *
import config
import os
import cv2
import einops
import numpy as np
import torch
import random
import pickle

from torch.utils.data import DataLoader
from pytorch_lightning import seed_everything
from cldm.model import create_model, load_state_dict
from cldm.ddim_hacked import DDIMSampler
from pathlib import Path

from omegaconf import OmegaConf
from ldm.util import instantiate_from_config
from PIL import Image
import matplotlib.pyplot as plt


from pytorch_lightning import seed_everything
from annotator.util import resize_image, HWC3
from annotator.openpose import OpenposeDetector
from cldm.model import create_model, load_state_dict
from cldm.ddim_hacked import DDIMSampler
from pathlib import Path

from omegaconf import OmegaConf
from ldm.util import instantiate_from_config
from PIL import Image
from torchvision import transforms as T
from tqdm import tqdm

In [2]:
config_file = './configs/pose_transfer.yaml'
model_ckpt = './lightning_logs/inshop_local_style/epoch=1-step=24952-v1.ckpt'


device = 'cuda:1'

batch_size = 2
ddim_steps = 20
eta = 0.0
scale = 9.0
H, W, C = 512, 512, 3
shape = (4, H // 8, W // 8)

n_prompt = 'longbody, lowres, bad anatomy, bad hands, missing fingers, extra digit, fewer digits, cropped, worst quality, low quality'
a_prompt = 'best quality, extremely detailed'

In [22]:
import re

def convert_fname(long_name):
    gender = 'MEN' if long_name[7:10]  == 'MEN' else 'WOMEN'

    input_list = long_name.replace('fashion','').split('___')
    
    # Define a regular expression pattern to match the relevant parts of each input string
    if gender == 'MEN':
        pattern = r'MEN(\w+)id(\d+)_(\d)(\w+)'
    else:
        pattern = r'WOMEN(\w+)id(\d+)_(\d)(\w+)'
    # Use a list comprehension to extract the matching substrings from each input string, and format them into the desired output format
    output_list = [f'{gender}/{category}/id_{id_num[:8]}/{id_num[8:]}_{view_num}_{view_desc}' for (category, id_num, view_num, view_desc) in re.findall(pattern, ' '.join(input_list))]

    # Print the resulting list of formatted strings
    return [f +'.jpg' for f in output_list]


def encode_style_images(full_styles_path):
    #full_styles_path = Path('./style_images')
    #style_names = ['face', 'hair', 'headwear', 'background', 'top', 'outer', 'bottom', 'shoes', 'accesories']
    style_embeddings = []
    for style_name in style_names:
        f_path = full_styles_path/f'{style_name}.jpg'
        #if not os.path.exists(str(f_path)):
        #    print(f'{str(f_path)} not found')                            
        style_image = style_encoder.preprocess(str(f_path)).to(device)
        style_emb = style_encoder.postprocess(style_encoder(style_image))
        style_embeddings.append(style_emb)
    styles = torch.tensor(np.array(style_embeddings)).squeeze(-2).unsqueeze(0).float().to(device)
    return styles

In [3]:
model = create_model(config_file).cpu()
model.load_state_dict(load_state_dict(model_ckpt, location=device))
model.cond_stage_model.device = device
model = model.to(device)

ddim_sampler = DDIMSampler(model)

No module 'xformers'. Proceeding without it.
ViscoNetLDM: Running in eps-prediction mode
DiffusionWrapper has 865.91 M params.
making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 4, 32, 32) = 4096 dimensions.
making attention of type 'vanilla' with 512 in_channels
Loaded model config from [./configs/pose_transfer.yaml]
Loaded state_dict from [./lightning_logs/inshop_local_style/epoch=1-step=24952-v1.ckpt]


In [16]:
proj_config = OmegaConf.load(config_file)
style_names = proj_config.dataset.train.params.style_names
style_encoder = instantiate_from_config(proj_config.model.style_embedding_config).to(device)

data_root = Path(proj_config.dataset.train.params.image_root)
image_root = data_root/proj_config.dataset.train.params.image_dir
style_root = data_root/proj_config.dataset.train.params.style_dir
pose_root = data_root/proj_config.dataset.train.params.pose_dir
mask_root = data_root/proj_config.dataset.train.params.mask_dir


In [11]:
#pose_image = np.array(Image.open('./test_samples/skeleton.png'))
#mask_image = np.array(Image.open('./test_samples/mask.png'))

In [59]:
def process(input_image, prompt, a_prompt, n_prompt, num_samples, image_resolution, detect_resolution, 
            ddim_steps, guess_mode, strength, scale, seed, eta, mask_enables, mask_image, pose_image, df_longname):    
    
    with torch.no_grad():
        src_name, dst_name = convert_fname(df_longname)
        
        # read pose and mask from dst
        import pdb

        pose_image = np.array(Image.open(str(pose_root/dst_name)))
        mask_image = Image.open(str(mask_root/dst_name).replace('.jpg','_mask.png'))
        
        mask = T.ToTensor()(T.Resize([image_resolution]*2, T.InterpolationMode.NEAREST)(mask_image))
        #mask = torch.tensor(mask_image.mean(-1)/255.,dtype=torch.float) #(512,512), [0,1]
        mask = mask.unsqueeze(0).to(device) # (1, 512, 512)
        input_image = np.array(Image.open(image_root/dst_name))
        # read style from src

        temp = src_name.replace('.jpg','').split('/')
        lastfolder = temp.pop(-1).replace('_','/', 1)
        style_folder = '/'.join(temp+[lastfolder])
        style_emb = encode_style_images(style_root/style_folder)
        src_image = np.array(Image.open(image_root/src_name))
        #mask = torch.tensor(mask_image.mean(-1)/255.,dtype=torch.float) #(512,512), [0,1]
        #mask = mask.unsqueeze(0).to(device) # (1, 512, 512)
        #style_emb = get_style_embeddings(style_folder).to(device)
        #style_emb = encode_style_images(style_root/src_name)

        input_image = HWC3(input_image)
        #detected_map, _ = apply_openpose(resize_image(input_image, detect_resolution))
        #detected_map = HWC3(detected_map)
        img = resize_image(input_image, image_resolution)
        H, W, C = img.shape

        # fix me
        detected_map = HWC3(pose_image)
        detected_map = cv2.resize(detected_map, (W, H), interpolation=cv2.INTER_NEAREST)

        control = torch.from_numpy(detected_map.copy()).float().to(device) / 255.0
        control = torch.stack([control for _ in range(num_samples)], dim=0)
        control = einops.rearrange(control, 'b h w c -> b c h w').clone()

        if seed == -1:
            seed = random.randint(0, 65535)
        seed_everything(seed)

        if config.save_memory:
            model.low_vram_shift(is_diffusing=False)

        cond = {"c_concat": [control], 
                "c_crossattn": [style_emb.repeat(num_samples, 1, 1, 1)],
                "c_text": [model.get_learned_conditioning([prompt + ', ' + a_prompt] * num_samples)],
                'c_concat_mask': [mask.repeat(num_samples, 1, 1, 1)]}

        un_cond = {"c_concat": [control], 
                   "c_crossattn": [torch.zeros_like(style_emb).repeat(num_samples, 1, 1, 1)],
                   "c_text":[model.get_learned_conditioning([n_prompt] * num_samples)],
                   'c_concat_mask': [torch.zeros_like(mask).repeat(num_samples, 1, 1, 1)]}

        shape = (4, H // 8, W // 8)

        if config.save_memory:
            model.low_vram_shift(is_diffusing=True)
        if guess_mode:
            model.control_scales = [strength * (0.825 ** float(12 - i)) for i in range(13)] if guess_mode else ([strength] * 13)  # Magic number. IDK why. Perhaps because 0.825**12<0.01 but 0.826**12>0.01
            print(model.control_scales)
        else:
            model.control_scales = [float(x) for x in strength.split(',')]
        model.mask_enables = [int(x) for x in mask_enables.split(',')]

        samples, intermediates = ddim_sampler.sample(ddim_steps, num_samples,
                                                     shape, cond, verbose=False, eta=eta,
                                                     unconditional_guidance_scale=scale,
                                                     unconditional_conditioning=un_cond)

        if config.save_memory:
            model.low_vram_shift(is_diffusing=False)

        x_samples = model.decode_first_stage(samples)
        x_samples = (einops.rearrange(x_samples, 'b c h w -> b h w c') * 127.5 + 127.5).cpu().numpy().clip(0, 255).astype(np.uint8)

        results = [Image.fromarray(x_samples[i]) for i in range(num_samples)]
        return results
        #ret_images = [np.array(mask_image), detected_map, src_image, input_image] +  results
        #return ret_images
        #crop 
        #return [img[:,80:-80,:] for img in ret_images]
        
    return 

In [70]:
input_image=None
a_prompt='best quality, extremely detailed'
n_prompt='longbody, lowres, bad anatomy, bad hands, missing fingers, extra digit, fewer digits, cropped, worst quality, low quality'
num_samples=5
image_resolution=512
detect_resolution=512
ddim_steps=20
guess_mode=False
scale=9.0
seed=-1
eta=0.0
mask_enables="1,1,1,1,1,1,1,1,1,1,1,1,1"
mask_image=None
pose_image=None
df_longname="fashionWOMENBlouses_Shirtsid0000035501_1front___fashionWOMENBlouses_Shirtsid0000035501_1front"

In [76]:
DEFAULT_BASELINE_STRENGTH = '0,0,0,0,0,0,0,0,0.5,0,0,0,0'

baseline_strength={
    #"realistic":"1,1,1,1,1,1,1,1,1,1,1,1,1",
    "realistic":"1,1,1,1,1,1,1,1,1,1,1,1,1",
    "picasso painting":DEFAULT_BASELINE_STRENGTH,
    "van gogh":DEFAULT_BASELINE_STRENGTH,
    "oil painting":DEFAULT_BASELINE_STRENGTH,
    "stained glass":DEFAULT_BASELINE_STRENGTH,
    "ukiyoe drawing":DEFAULT_BASELINE_STRENGTH,
    "computer graphics":DEFAULT_BASELINE_STRENGTH,
    "cyberpunk japanese anime":DEFAULT_BASELINE_STRENGTH,
    "grunge street art":DEFAULT_BASELINE_STRENGTH,
    'color sketch':DEFAULT_BASELINE_STRENGTH,
    'Disney cartoon':DEFAULT_BASELINE_STRENGTH,     
}

DEFAULT_CLOTHING_STRENGTH = '0.1,0.1,0.1,0.2,0.2,0.2,0.2,0.3,0.4,0,0,0,0'
clothing_strength={
    #"realistic":"1,1,1,1,1,1,1,1,1,1,1,1,1",
    "realistic":DEFAULT_CLOTHING_STRENGTH,
    "picasso painting":'0,0,0.0,0.5,0.5,0.5,0.5,0.5,0.5,0,0,0,0,0',
    "van gogh":'0,0,0.0,0.5,0.5,0.5,0.5,0.5,0.5,0,0,0,0,0',
    "oil painting":'0,0,0.0,0.5,0.5,0.5,0.5,0.5,0.5,0,0,0,0,0',
    "stained glass":'1,1,1,0.5,0.5,0.5,0.5,0.5,0.5,0,0,0,0,0',
    "ukiyoe drawing":'1,1,1,1,1,0.5,0.5,0.5,0.5,0,0,0,0,0',
    "computer graphics":DEFAULT_CLOTHING_STRENGTH,
    "cyberpunk japanese anime":DEFAULT_CLOTHING_STRENGTH,
    "grunge street art":DEFAULT_CLOTHING_STRENGTH,
    'color sketch':DEFAULT_CLOTHING_STRENGTH,
    'Disney cartoon':DEFAULT_CLOTHING_STRENGTH,     
}


image_styles = list(clothing_strength.keys())
person_texts=["a woman, ",
              "a woman, standing in a jungle"]
subfolders=["baseline", "mode"]


In [77]:
output_dir = Path('./test_output_control/')
total_samples = 100
n_loops = total_samples//num_samples

In [78]:
for image_style in tqdm(image_styles[:]):
    for sub_id, subfolder in enumerate(subfolders[1:]):
        s = image_style.replace(' ','_')
        save_root = output_dir/s/subfolder
        os.makedirs(str(save_root), exist_ok=True)
        save_id = 0
        prompt = person_texts[sub_id]+','+image_style
        print(prompt)
        if subfolder=='baseline':
            strength = baseline_strength[image_style]
        else:
            strength = clothing_strength[image_style]            
        for _ in range(n_loops):
            images = process(input_image, prompt, a_prompt, n_prompt, num_samples, image_resolution, detect_resolution, ddim_steps, guess_mode, strength, scale, seed, eta, mask_enables, mask_image, pose_image, df_longname)            
            for k, image in enumerate(images):
                
                fname = save_root/f'{save_id}.png'
                image.save(fname)           
                save_id+=1


Global seed set to 358


realistic
a woman, ,realistic
Data shape for DDIM sampling is (5, 4, 64, 64), eta 0.0
Running DDIM Sampling with 20 timesteps


DDIM Sampler: 100%|█| 20/20 [00:15<00:
Global seed set to 31411


Data shape for DDIM sampling is (5, 4, 64, 64), eta 0.0
Running DDIM Sampling with 20 timesteps


DDIM Sampler: 100%|█| 20/20 [00:15<00:
Global seed set to 57934


...

DDIM Sampler: 100%|█| 20/20 [00:16<00:
Global seed set to 33281


Data shape for DDIM sampling is (5, 4, 64, 64), eta 0.0
Running DDIM Sampling with 20 timesteps


DDIM Sampler: 100%|█| 20/20 [00:16<00:
Global seed set to 9181


Data shape for DDIM sampling is (5, 4, 64, 64), eta 0.0
Running DDIM Sampling with 20 timesteps


DDIM Sampler: 100%|█| 20/20 [00:16<00:
